In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\ml-with-MLFlow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/phuthinhnhpt123/ml-with-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="phuthinhnhpt123"
os.environ["MLFLOW_TRACKING_PASSWORD"]="258203de851984e7040704678c76c12bd899f3b1"

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [31]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [37]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [33]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/phuthinhnhpt123/ml-with-MLFlow.mlflow",
        )

        return model_evaluation_config

In [34]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import json

In [35]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)

        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rsme, mae, r2) = self.eval_metrics(test_y,predicted_qualities)

            scores={"rmse": rsme, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rsme)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticModel")
            else:
                mlflow.sklearn.log_model(model,"model")

In [36]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-04-07 17:32:15,332: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-07 17:32:15,354: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-07 17:32:15,360: INFO: common: yaml file: schema.yaml loaded successfully]


[2024-04-07 17:32:15,364: INFO: common: created directory at: artifacts]
[2024-04-07 17:32:15,365: INFO: common: created directory at: artifacts/model_evaluation]


c:\Users\Admin\miniconda3\envs\mlprj\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Registered model 'ElasticModel' already exists. Creating a new version of this model...
2024/04/07 17:32:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticModel, version 2
Created version '2' of model 'ElasticModel'.
